In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('all_data.csv')

Исследуем данные.

In [4]:
print "пациентов: {0}".format(data.shape[0])
print "признаков: {0}".format(data.shape[1])

пациентов: 129
признаков: 108


Это тот набор фичей, который выделил врач, который предоставил данные. <br\>
Если я все правильно понимаю, нужно научиться выбирать правильную первую линию ХТ ("Схема лечения (1 линия ХТ)") <br>
Характеристикой качества работы лечения видимо выступает время жизни пациентов (от наблюдения до момента закрытия карты), эти данные не обновлены. <br>
Перевая линия ХТ прописана у 24 пациентов из 129, у остальных пропуски. <br>


In [24]:
main_features = [
    "Код",
    "Возраст",
    "Пол",
    "Дата установления диагноза",
    "Сопутствующая патология",
    "Стадия",
    "Желчеразгрузочная операция",
    "Дата желчеразгрузочной операции",
    "Гистология опухоли (до лечения)",
    "ECOG (до лечения)",
    "Вес (до лечения)",
    "ИМТ (до лечения)",
    "Общ белок (до лечения)",
    "АЛТ (до лечения)",
    "АСТ (до лечения)",
    "ЛДГ (до лечения)",
    "CA 19-9 (до лечения)",
    "РЭA (до лечения)",
    "Схема лечения (1 линия ХТ)",
    "Дата начала (1 линия ХТ)",
    "Диагноз (обновленный",
    "Дата последнего наблюдения",
    "Карта закрыта"
]

In [25]:
main_data = data[main_features]

Вот так выглядит статистика по первой линии ХТ, т.е. два основных препарата, пропусков 82%

In [26]:
main_data['Схема лечения (1 линия ХТ)'].value_counts()

Gemcitabin                                                                                                                                                     8
FOLFIRINOX                                                                                                                                                     8
FOLFOX                                                                                                                                                         2
гемзар                                                                                                                                                         1
Gemcitabin + cisplatin                                                                                                                                         1
Gemcitabin + паклитаксел                                                                                                                                       1
AF                                

# Выделим целевую переменную для тех, для кого известна первая линия ХТ

In [71]:
main_data = main_data.rename(columns={'Дата последнего наблюдения': 'last_date',
                                      'Дата установления диагноза': 'first_date',
                                      'Карта закрыта': 'closed',
                                     })

In [181]:
def create_mask(column_name):
    anti_mask = np.array(main_data[column_name].isnull())
    mask = np.logical_not(anti_mask)
    return mask

In [147]:
anti_healing_mask = np.array(main_data['Схема лечения (1 линия ХТ)'].isnull())
healing_mask = np.logical_not(anti_healing_mask)

In [182]:
healing_mask = create_mask('Схема лечения (1 линия ХТ)')
first_date_mask = create_mask('first_date')
last_date_mask = create_mask('last_date')

In [183]:
mask = np.logical_and(healing_mask,last_date_mask)

In [180]:
main_data[last_date_mask].loc[:,'last_date'].isnull()

0       True
1       True
2       True
3      False
9      False
21     False
22     False
30     False
41     False
54     False
62     False
71     False
79     False
81      True
93     False
96     False
98     False
99     False
105    False
108    False
117    False
118    False
126    False
128     True
Name: last_date, dtype: bool

In [184]:
main_data[mask]['last_date']

3      16.07.16 00:00
9      10.04.15 00:00
21     17.05.16 00:00
22     03.06.15 00:00
30     29.12.15 00:00
41     03.02.15 00:00
54     30.12.15 00:00
62     13.08.15 00:00
71     26.08.15 00:00
79     07.10.15 00:00
93     28.03.16 00:00
96     11.03.15 00:00
98     31.07.15 00:00
99     22.03.16 00:00
105    06.11.15 00:00
108    16.06.15 00:00
117    08.06.15 00:00
118    09.05.16 00:00
126    06.03.15 00:00
Name: last_date, dtype: object

In [32]:
main_data[mask].to_csv('raw_masked.csv')

 Превратим строки в даты посчитаем всякие разности.

In [153]:
type(main_data[mask]['first_date'][0])

str

In [154]:
from datetime import datetime

def dt_from_str(date_time_string):
    if date_time_string is not np.nan:
        return datetime.strptime(date_time_string, '%d.%m.%y %H:%M')
    else:
        return np.nan

In [188]:
first_date = np.array(map(dt_from_str, main_data[mask]['first_date']))
last_date = np.array(map(dt_from_str, main_data[mask]['last_date']))


### Времена жизни

In [204]:
(last_date - first_date)

array([datetime.timedelta(326), datetime.timedelta(212),
       datetime.timedelta(783), datetime.timedelta(117),
       datetime.timedelta(5), datetime.timedelta(12),
       datetime.timedelta(142), datetime.timedelta(3),
       datetime.timedelta(79), datetime.timedelta(379),
       datetime.timedelta(383), datetime.timedelta(8),
       datetime.timedelta(14), datetime.timedelta(517),
       datetime.timedelta(29), datetime.timedelta(252),
       datetime.timedelta(74), datetime.timedelta(146),
       datetime.timedelta(29)], dtype=object)

In [140]:
type(last_date[0])

datetime.datetime

In [195]:
main_data[mask]['Схема лечения (1 линия ХТ)'].value_counts()

Gemcitabin                                                                                                                                                     8
FOLFIRINOX                                                                                                                                                     4
FOLFOX                                                                                                                                                         2
гемзар                                                                                                                                                         1
Gemcitabin + паклитаксел                                                                                                                                       1
AF                                                                                                                                                             1
Gemcitabin + Docetaxel            

In [200]:
main_data[mask].head()

,Код,Возраст,Пол,first_date,Сопутствующая патология,Стадия,Желчеразгрузочная операция,Дата желчеразгрузочной операции,Гистология опухоли (до лечения),ECOG (до лечения),...,АЛТ (до лечения),АСТ (до лечения),ЛДГ (до лечения),CA 19-9 (до лечения),РЭA (до лечения),Схема лечения (1 линия ХТ),Дата начала (1 линия ХТ),Диагноз (обновленный,last_date,closed
3,5,60,Ж,25.08.15 00:00,нет,не установлена,нет,NaN,умереннодифференцированная протоковая аденокар...,1,...,"8,9","15,1",380,264,7,FOLFIRINOX,08.10.15 00:00,Сочетанное поражение,16.07.16 00:00,False
9,11,66,Ж,10.09.14 00:00,Сахарный диабет 2 типа,не установлена,не было,NaN,умереннодифференцированная аденокарцинома,1,...,13,16,NaN,12245,6,Gemcitabin,27.11.14 00:00,Рак хвоста,10.04.15 00:00,True
21,23,50,М,26.03.14 00:00,"ГБ 2ст. Сах. Диабет 2 типа, хронич панкреотит",IIb,NaN,NaN,"умереннодифф аденокорценома, без признаков опу...",2,...,20,20,NaN,10,NaN,Gemcitabin,04.04.14 00:00,Рак головки,17.05.16 00:00,False
22,24,55,М,06.02.15 00:00,"Рак головки поджелудочной железы pT4NxM1, MTS...",не установлена,NaN,NaN,ткань печени - аденокорценома протокового типа,2,...,"4,1","6,5",NaN,NaN,NaN,FOLFOX,06.03.15 00:00,NaN,03.06.15 00:00,False
30,32,67,М,24.12.15 00:00,NaN,III,NaN,NaN,умереннодифференцированная аденокарцинома (опу...,1,...,46,46,224,NaN,NaN,FOLFOX,10.11.15 00:00,Рак головки,29.12.15 00:00,False


In [86]:
for col in main_data.columns:
    print "{0}: {1} {2}%".format(col, (30 - (len(col) +  col.count(' '))/2)*" ", 1 - float(main_data[col].count()) / main_data[col].size)

Код:                             0.0%
Возраст:                         0.046511627907%
Пол:                             0.0%
Дата установления диагноза:      0.062015503876%
Сопутствующая патология:         0.232558139535%
Стадия:                          0.0697674418605%
Желчеразгрузочная операция:      0.62015503876%
Дата желчеразгрузочной операции:  0.651162790698%
Гистология опухоли (до лечения):  0.348837209302%
ECOG (до лечения):                  0.093023255814%
Вес (до лечения):                 0.077519379845%
ИМТ (до лечения):                 0.093023255814%
Общ белок (до лечения):           0.077519379845%
АЛТ (до лечения):                 0.093023255814%
АСТ (до лечения):                 0.093023255814%
ЛДГ (до лечения):                 0.961240310078%
CA 19-9 (до лечения):                0.759689922481%
РЭA (до лечения):                  0.790697674419%
Схема лечения (1 линия ХТ):        0.813953488372%
Дата начала (1 линия ХТ):          0.813953488372%
Диагноз (обновленный:

In [19]:
anti_mask = np.array(main_data['Схема лечения (1 линия ХТ)'].isnull())
mask = np.logical_not(anti_mask)

Вот так выглядят пропуски, в той части данных где целевая переменная не пропуск

In [129]:
for col in main_data[mask].columns:
     print "{0}: {1} {2}%".format(col, (30 - (len(col) +  col.count(' '))/2)*" ", 1 - float(main_data[mask][col].count()) / main_data[mask][col].size)

Код:                             0.0%
Возраст:                         0.0%
Пол:                             0.0%
Дата установления диагноза:      0.0%
Сопутствующая патология:         0.25%
Стадия:                          0.0%
Желчеразгрузочная операция:      0.5%
Дата желчеразгрузочной операции:  0.583333333333%
Гистология опухоли (до лечения):  0.166666666667%
ECOG (до лечения):                  0.0%
Вес (до лечения):                 0.0%
ИМТ (до лечения):                 0.0%
Общ белок (до лечения):           0.0%
АЛТ (до лечения):                 0.0833333333333%
АСТ (до лечения):                 0.0833333333333%
ЛДГ (до лечения):                 0.833333333333%
CA 19-9 (до лечения):                0.25%
РЭA (до лечения):                  0.375%
Схема лечения (1 линия ХТ):        0.0%
Дата начала (1 линия ХТ):          0.0416666666667%
Диагноз (обновленный:             0.166666666667%
